
```
conda create -n tf_env python=3.10 pip -y
conda activate tf_env
pip install tensorflow==2.17.1 keras==3.5.0 numpy==1.26.4 scipy==1.13.1 seaborn==0.13.2 tensorflow-model-optimization==0.8.0 scikit-plot==0.3.7 tf_keras==2.17.0
python3 -m pip install tensorflow[and-cuda]
```

In [14]:
# ! pip install tensorflow-model-optimization

Imports

In [15]:
import tensorflow as tf
import numpy as np
import scipy
import os
from tensorflow_model_optimization.python.core.keras.compat import keras
import time 

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {scipy.__version__}")



TensorFlow version: 2.17.1
NumPy version: 1.26.4
SciPy version: 1.13.1


In [16]:
if tf.config.list_physical_devices('GPU'):
    print("GPU está conectada")
    gpus = tf.config.list_physical_devices('GPU')
    for gpu in gpus:
        print(f"Nome da GPU: {gpu.name}")
else:
    print("Não conectado a uma GPU")

GPU está conectada
Nome da GPU: /physical_device:GPU:0


Funções auxiliares

In [17]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, average_precision_score
from scipy.stats import ks_2samp
#import scikitplot as skplt
import matplotlib.pyplot as plt
from numpy import interp
from sklearn.metrics import mean_squared_error
import seaborn as sns


from scipy.stats import ks_2samp
from sklearn.metrics import roc_curve, auc


def extract_final_losses(history):
    """Função para extrair o melhor loss de treino e validação.

    Argumento(s):
    history -- Objeto retornado pela função fit do keras.

    Retorno:
    Dicionário contendo o melhor loss de treino e de validação baseado
    no menor loss de validação.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    idx_min_val_loss = np.argmin(val_loss)
    return {'train_loss': train_loss[idx_min_val_loss], 'val_loss': val_loss[idx_min_val_loss]}

def plot_training_error_curves(history):
    """Função para plotar as curvas de erro do treinamento da rede neural.

    Argumento(s):
    history -- Objeto retornado pela função fit do keras.

    Retorno:
    A função gera o gráfico do treino da rede e retorna None.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    fig, ax = plt.subplots()
    ax.plot(train_loss, label='Train')
    ax.plot(val_loss, label='Validation')
    ax.set(title='Training and Validation Error Curves', xlabel='Epochs', ylabel='Loss (MSE)')
    ax.legend()
    plt.show()

def compute_performance_metrics(y, y_pred_class, y_pred_scores=None):
    accuracy = accuracy_score(y, y_pred_class)
    recall = recall_score(y, y_pred_class)
    precision = precision_score(y, y_pred_class)
    f1 = f1_score(y, y_pred_class)
    performance_metrics = (accuracy, recall, precision, f1)
    if y_pred_scores is not None:
        # skplt.metrics.plot_ks_statistic(y, y_pred_scores)
        # plt.savefig("ks_plot.png")
        # plt.show()
        y_pred_scores = y_pred_scores[:, 1]
        auroc = roc_auc_score(y, y_pred_scores)
        aupr = average_precision_score(y, y_pred_scores)
        performance_metrics = performance_metrics + (auroc, aupr)
    return performance_metrics

def print_metrics_summary(accuracy, recall, precision, f1, auroc=None, aupr=None):
    print()
    print("{metric:<18}{value:.4f}".format(metric="Accuracy:", value=accuracy))
    print("{metric:<18}{value:.4f}".format(metric="Recall:", value=recall))
    print("{metric:<18}{value:.4f}".format(metric="Precision:", value=precision))
    print("{metric:<18}{value:.4f}".format(metric="F1:", value=f1))
    if auroc is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUROC:", value=auroc))
    if aupr is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUPR:", value=aupr))

Definindo paths e carregabdo os dados

In [18]:
path = "/home/pedro/projetoDL/dataset/processado/"
save_path = "/home/pedro/projetoDL/log/prunning/new-prunning/"

# Verifica se o diretório existe, se não, cria o diretório
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Carregar os dados
Y = np.load(path + 'Y_train_NewApproach_Injected_v2.npz')
Y= Y.f.arr_0

X = np.load(path + 'X_train_NewApproach_Injected_v2.npz')
X = X.f.arr_0

Carregar os dados de teste

In [19]:
# path = "/home/pedro/projetoDL/dataset/processado/"

# Y = np.load(path + 'Y_test_Driving_NewApproach_Injected_v2.npz')
# Y= Y.f.arr_0

# X = np.load(path + 'X_test_Driving_NewApproach_Injected_v2.npz')
# X = X.f.arr_0

# from sklearn.model_selection import train_test_split

# # Definir 2/3 dos dados
# X_subset, _, Y_subset, _ = train_test_split(
#     X, Y, train_size=1/3, stratify=Y, random_state=42  # Stratify para manter a proporção de classes
# )

# # Agora X_subset e Y_subset têm 2/3 dos dados originais
# X_test = X_subset
# Y_test = Y_subset

Selecionar os dados do fold 3 (Melhor resultado)

In [20]:
import tensorflow_model_optimization as tfmot
from sklearn.model_selection import KFold
import numpy as np
import os

# Configurar o KFold e selecionar o fold 3
skf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 0

for train_index, val_index in skf.split(X, Y):
    fold_no += 1
    if fold_no == 3:  # Selecionar o fold 3
        x_train, x_val = X[train_index], X[val_index]
        y_train, y_val = Y[train_index], Y[val_index]
        break



# Verificar os dados
print(f"Tamanho de x_train: {x_train.shape}, y_train: {y_train.shape}")
print(f"Tamanho de x_val: {x_val.shape}, y_val: {y_val.shape}")
print("Distribuição em y_train:", np.unique(y_train, return_counts=True))
print("Distribuição em y_val:", np.unique(y_val, return_counts=True))

KeyboardInterrupt: 

In [ ]:
import warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.get_logger().setLevel("ERROR")
warnings.simplefilter("ignore")

# Treinar o modelo sem prunning

In [ ]:
from tensorflow_model_optimization.python.core.keras.compat import keras
from PIL import Image

# Configurar o mixed precision
# Equivalent to the two lines above
keras.mixed_precision.set_global_policy('float32')

# Definir o modelo Sequential
model = keras.Sequential([
    # 1ª camada convolucional
    keras.layers.Conv2D(filters=32, kernel_size=5, strides=(1, 1), activation='relu',
                        kernel_regularizer=keras.regularizers.l2(0.01), padding='same', input_shape=(44, 116, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # 2ª camada convolucional
    keras.layers.Conv2D(filters=64, kernel_size=5, strides=(1, 1), activation='relu',
                        kernel_regularizer=keras.regularizers.l2(0.01), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Camada de Flatten para transformar os dados em vetor
    keras.layers.Flatten(),
    keras.layers.Dropout(0.3),

    # Camadas densas (fully connected)
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid', dtype='float32')  # Saída para classificação binária
])

# Mostrar resumo do modelo
print(model.summary())

# Gerar visualização do modelo
# keras.utils.plot_model(model, to_file="conv2d.png", show_shapes=True, show_layer_names=True)
# display(Image.open('conv2d.png'))

# Compilando o modelo
model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"]
)

# Callbacks
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.00001
)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Treinando o modelo
batch_size = 256
history = model.fit(
    x_train, y_train,
    epochs=30,
    validation_data=(x_val, y_val),
    batch_size=batch_size,
    callbacks=[learning_rate_reduction, early_stopping],
    verbose=1,
    shuffle=True
)


In [ ]:
model_save_path = save_path + "full/"

# Verifica se o diretório existe, se não, cria o diretório
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)


keras_file = model_save_path + 'original_model.h5'
keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved full baseline model to:', keras_file)

Avaliar o desempenho e salvar o modelo pra uso posterior

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
# import scikitplot as skplt

file_name = f'log_original_model.txt'
file1 = open(os.path.join(model_save_path, file_name), "a")

start = time.time()
y_pred_scores = model.predict(x_val)
end = time.time()

print(f"Runtime of the program is {end - start}", file=file1)
total_time = end - start
timesample = total_time/y_pred_scores.shape[0]
print(f"us/sample is {timesample*1000000}", file=file1)

y_pred_class = (y_pred_scores > 0.5).astype("int64")

y_pred_scores2 = y_pred_scores
y_pred_scores_0 = 1 - y_pred_scores
y_pred_scores = np.concatenate([y_pred_scores_0, y_pred_scores], axis=1)

accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_val, y_pred_class, y_pred_scores)

mseTeste = mean_squared_error(y_val, y_pred_class)

print(f'Results for original_model: Recall of {recall}; accuracy of {accuracy}; precision of {precision}; f1 of {f1}; auroc of {auroc}; aupr of {aupr}', file=file1)

sns.set(rc={'figure.figsize':(8,8)})
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.savefig(model_save_path + "loss_and_val_loss.png")
plt.show()

sns.set(rc={'figure.figsize':(8,8)})
plt.figure(figsize=(8,4))
plt.plot(history.history['accuracy'], label='Train accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.savefig(model_save_path + "loss_and_val_accuracy.png")
plt.show()

fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_val, y_pred_scores2)
auc_keras = auc(fpr_keras, tpr_keras)


sns.set(rc={'figure.figsize':(8,8)})
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
plt.plot(fpr_keras, tpr_keras, marker='.', label='MLP (auc = %0.3f)' % auc_keras)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig(model_save_path + "plot_roc.png")
plt.show()
#########################################################################################################
#Plotando o histórico do treino
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

file1.close()

# Carregando o modelo se nao carregar

In [ ]:
# from tensorflow_model_optimization.python.core.keras.compat import keras

# Carregar o modelo
model_path = '/home/pedro/projetoDL/log/prunning/new-prunning/full/original_model.h5'
model = keras.models.load_model(model_path)

I0000 00:00:1732584558.815743 1169999 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732584558.815890 1169999 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732584558.815943 1169999 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732584558.914806 1169999 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

# Fine-tune pre-trained model with prunning

In [ ]:
import tensorflow_model_optimization as tfmot
import numpy as np

# Pruning: Reduzir a magnitude de pesos irrelevantes no modelo
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Definir parâmetros de pruning
batch_size = 256
epochs = 6
validation_split = 0.1  # Usar 10% do conjunto de treinamento para validação

# Calcular o número de passos para finalizar o pruning após 6 épocas
num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Definir parâmetros de pruning
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.50,  # Começar com 50% de pesos zerados
        final_sparsity=0.90,    # Finalizar com 80% de pesos zerados
        begin_step=0,
        end_step=end_step
    )
}

# Envolver o modelo com poda
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
model_for_pruning = prune_low_magnitude(model, **pruning_params)

# Compilar o modelo
model_for_pruning.compile(
    optimizer=keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Treinar o modelo com callbacks de poda
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
]

# Treinar o modelo com pruning
history_pruning = model_for_pruning.fit(
    x_train, y_train,
    epochs=epochs,
    validation_split=0.1,
    batch_size=batch_size,
    callbacks=callbacks,
    verbose=1,
    shuffle=True
)


In [ ]:
model_prunned_save_path = save_path + "prunned/"

# # Verifica se o diretório existe, se não, cria o diretório
# if not os.path.exists(model_prunned_save_path):
#     os.makedirs(model_prunned_save_path)


# keras_file = model_prunned_save_path + 'prunned.h5'
# keras.models.save_model(model_for_pruning, keras_file, include_optimizer=False)
# print('Saved prunned baseline model to:', keras_file)


# Remover wrappers de poda para exportação
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# Salvar como H5 para GPU
model_for_export.save(model_prunned_save_path + "model_pruned_gpu.h5")

# Modelos Ja treinados (carregar)

In [ ]:
model_path = '/home/pedro/projetoDL/log/prunning/new-prunning/prunned/model_pruned_gpu.h5'
model_for_pruning = keras.models.load_model(model_path)

In [ ]:
# Calcular o número de parâmetros para o modelo original (model)
# trainable_params_model = sum(np.prod(weight.shape) for weight in model.trainable_weights)
# non_trainable_params_model = sum(np.prod(weight.shape) for weight in model.non_trainable_weights)

# print(f"Modelo original (model):")
# print(f"Parâmetros treináveis: {trainable_params_model}")
# print(f"Parâmetros não treináveis: {non_trainable_params_model}")

# Calcular o número de parâmetros para o modelo com pruning (model_for_pruning)
trainable_params_pruning = sum(np.prod(weight.shape) for weight in model_for_pruning.trainable_weights)
non_trainable_params_pruning = sum(np.prod(weight.shape) for weight in model_for_pruning.non_trainable_weights)

print(f"\nModelo com pruning (model_for_pruning):")
print(f"Parâmetros treináveis: {trainable_params_pruning}")
print(f"Parâmetros não treináveis: {non_trainable_params_pruning}")


Modelo com pruning (model_for_pruning):
Parâmetros treináveis: 1359041
Parâmetros não treináveis: 192


In [ ]:
total_params = sum(np.prod(weight.shape) for weight in model_for_pruning.trainable_weights)
nonzero_params = sum(np.count_nonzero(weight.numpy()) for weight in model_for_pruning.trainable_weights)

sparsity = 1 - (nonzero_params / total_params)
print(f"Sparsidade do modelo: {sparsity * 100:.2f}%")

Sparsidade do modelo: 89.98%


In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)


In [ ]:
# Função para calcular o número de parâmetros treináveis
def get_model_params(model):
    return np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])

# Calcular o número de parâmetros antes e depois da poda
params_before_pruning = get_model_params(model)
params_after_pruning = get_model_params(model_for_export)

# Mostrar os resultados
print(f"Parâmetros antes da poda: {params_before_pruning}")
print(f"Parâmetros após a poda: {params_after_pruning}")
print(f"Parâmetros removidos: {params_before_pruning - params_after_pruning} "
      f"({(params_before_pruning - params_after_pruning) / params_before_pruning * 100:.2f}%)")


Parâmetros antes da poda: 1359041
Parâmetros após a poda: 1359041
Parâmetros removidos: 0 (0.00%)


# Evaluate prunned model

In [ ]:
model_prunned_save_path = save_path + "prunned/"

file_name = f'log_modelo_fold3.txt'
file1 = open(os.path.join(model_prunned_save_path, file_name), "a")

start = time.time()
y_pred_scores = model_for_pruning.predict(x_val)
end = time.time()

print(f"Runtime of the program is {end - start}", file=file1)
total_time = end - start
timesample = total_time/y_pred_scores.shape[0]

y_pred_class = (y_pred_scores > 0.5).astype("int64")

y_pred_scores2 = y_pred_scores
y_pred_scores_0 = 1 - y_pred_scores
y_pred_scores = np.concatenate([y_pred_scores_0, y_pred_scores], axis=1)

accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_val, y_pred_class, y_pred_scores)


mseTeste = mean_squared_error(y_val, y_pred_class)

print(f'Results for Prunned Model [89% sparsity]:\n Recall of {recall}; accuracy of {accuracy}; precision of {precision}; f1 of {f1}; auroc of {auroc}; aupr of {aupr}', file=file1)
print(f"us/sample is {timesample*1000000}", file=file1)


# sns.set(rc={'figure.figsize':(8,8)})
# plt.figure(figsize=(8,4))
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epochs')
# plt.legend(loc='upper right')
# plt.savefig(model_prunned_save_path + "loss_and_val_loss_prunned.png")
# plt.show()

# sns.set(rc={'figure.figsize':(8,8)})
# plt.figure(figsize=(8,4))
# plt.plot(history.history['accuracy'], label='Train accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation accuracy')
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epochs')
# plt.legend(loc='upper right')
# plt.savefig(model_prunned_save_path + "loss_and_val_accuracy_prunned.png")
# plt.show()

# sns.set(rc={'figure.figsize':(8,8)})
# subplot = skplt.metrics.plot_confusion_matrix(y_val, y_pred_class, normalize=True)
# subplot.set_ylim(-0.5, 1.5)
# plt.savefig(save_path + "conf_matrix.png")
# plt.show()

fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_val, y_pred_scores2)
auc_keras = auc(fpr_keras, tpr_keras)


sns.set(rc={'figure.figsize':(8,8)})
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
plt.plot(fpr_keras, tpr_keras, marker='.', label='MLP (auc = %0.3f)' % auc_keras)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig(model_prunned_save_path + "plot_roc_prunned.png")
plt.show()
#########################################################################################################
#Plotando o histórico do treino
# pd.DataFrame(history.history).plot(figsize=(8, 5))
# plt.grid(True)
# plt.gca().set_ylim(0, 1)
# plt.show()

file1.close()

Create a 3x smaller models from pruning

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

model_3xprunned_save_path = save_path + "prunned/"

# Verifica se o diretório existe, se não, cria o diretório
if not os.path.exists(model_prunned_save_path):
    os.makedirs(model_prunned_save_path)

pruned_keras_file = model_3xprunned_save_path + '3xPrunned.h5'
keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

In [ ]:
model_prunned_save_path = save_path + "prunned/"

file_name = f'log_modelo_fold3.txt'
file1 = open(os.path.join(model_prunned_save_path, file_name), "a")

start = time.time()
y_pred_scores = model_for_export.predict(x_val)
end = time.time()

print(f"Runtime of the program is {end - start}", file=file1)
total_time = end - start
timesample = total_time/y_pred_scores.shape[0]

y_pred_class = (y_pred_scores > 0.5).astype("int64")

y_pred_scores2 = y_pred_scores
y_pred_scores_0 = 1 - y_pred_scores
y_pred_scores = np.concatenate([y_pred_scores_0, y_pred_scores], axis=1)

accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_val, y_pred_class, y_pred_scores)


mseTeste = mean_squared_error(y_val, y_pred_class)

print(f'\nResults for Prunned Model [89% sparsity - 3x prunned]:\n Recall of {recall}; accuracy of {accuracy}; precision of {precision}; f1 of {f1}; auroc of {auroc}; aupr of {aupr}', file=file1)
print(f"us/sample is {timesample*1000000}", file=file1)


fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_val, y_pred_scores2)
auc_keras = auc(fpr_keras, tpr_keras)


sns.set(rc={'figure.figsize':(8,8)})
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
plt.plot(fpr_keras, tpr_keras, marker='.', label='MLP (auc = %0.3f)' % auc_keras)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig(model_prunned_save_path + "plot_roc_prunned.png")
plt.show()


file1.close()

3x prunned tflite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

pruned_tflite_file = model_3xprunned_save_path + 'TFLite_3xPrunned.tflite'

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

In [ ]:
# Função para calcular o número de parâmetros treináveis
def get_model_params(model):
    return np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])

# Calcular o número de parâmetros antes e depois da poda
params_before_pruning = get_model_params(model)
params_after_pruning = get_model_params(model_for_export)

# Mostrar os resultados
print(f"Parâmetros antes da poda: {params_before_pruning}")
print(f"Parâmetros após a poda: {params_after_pruning}")
print(f"Parâmetros removidos: {params_before_pruning - params_after_pruning} "
      f"({(params_before_pruning - params_after_pruning) / params_before_pruning * 100:.2f}%)")


evaluate prunned tflite

In [ ]:
import numpy as np

# Salvar uma pequena parte dos dados
small_x_val = x_val[:100]  # Pegando as 100 primeiras amostras
small_y_val = y_val[:100]  # Pegando os rótulos correspondentes

np.save("small_x_val.npy", small_x_val)
np.save("small_y_val.npy", small_y_val)

# Modelo TFLite já está em `pruned_tflite_file`


In [ ]:
# import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_curve, auc, mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

# Carregar o modelo TFLite
# interpreter = tf.lite.Interpreter(model_path=pruned_tflite_file)
#interpreter.allocate_tensors()
# Criar o interpretador com múltiplos threads
interpreter = tf.lite.Interpreter(
    model_path=pruned_tflite_file,
    num_threads=12  # Defina o número desejado de threads
)
interpreter.allocate_tensors()


# Pegar detalhes do tensor de entrada e saída
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Certifique-se de que x_val está no formato esperado
x_val_tflite = np.expand_dims(x_val, axis=-1)  # Adicionar dimensão do canal
x_val_tflite = x_val_tflite.astype(np.float32)  # Garantir que o tipo é float32

# Fazer previsões com o modelo TFLite
y_pred_scores_tflite = []
start = time.time()

for sample in x_val_tflite:
    sample = np.expand_dims(sample, axis=0)  # Adicionar dimensão batch
    interpreter.set_tensor(input_details[0]['index'], sample)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_details[0]['index'])
    y_pred_scores_tflite.append(prediction)

end = time.time()
y_pred_scores_tflite = np.array(y_pred_scores_tflite).squeeze()

# Calcular o tempo médio por amostra
total_time_tflite = end - start
timesample_tflite = total_time_tflite / y_pred_scores_tflite.shape[0]

# Classe prevista
y_pred_class_tflite = (y_pred_scores_tflite > 0.5).astype("int64")

# Ajustar as previsões para um formato bidimensional
y_pred_scores_tflite = np.expand_dims(y_pred_scores_tflite, axis=1)  # Torna bidimensional: [N] -> [N, 1]
y_pred_scores_tflite_0 = 1 - y_pred_scores_tflite  # Probabilidades da classe 0

# Concatenar probabilidades para cada classe
y_pred_scores_tflite_concat = np.concatenate([y_pred_scores_tflite_0, y_pred_scores_tflite], axis=1)

# Calcular métricas
accuracy_tflite, recall_tflite, precision_tflite, f1_tflite, auroc_tflite, aupr_tflite = compute_performance_metrics(
    y_val, y_pred_class_tflite, y_pred_scores_tflite_concat
)
mseTeste_tflite = mean_squared_error(y_val, y_pred_class_tflite)

# Exibir os resultados
print(f"Runtime of the TFLite model is {total_time_tflite}")
print(f"Time per sample is {timesample_tflite * 1e6:.2f} us/sample")
print(f"\nResults for Pruned TFLite Model:\n Recall: {recall_tflite}; Accuracy: {accuracy_tflite}; Precision: {precision_tflite}; F1: {f1_tflite}; AUROC: {auroc_tflite}; AUPR: {aupr_tflite}")

# Curva ROC
fpr_tflite, tpr_tflite, thresholds_tflite = roc_curve(y_val, y_pred_scores_tflite)
auc_tflite = auc(fpr_tflite, tpr_tflite)

sns.set(rc={'figure.figsize': (8, 8)})
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='black')
plt.plot(fpr_tflite, tpr_tflite, marker='.', label='TFLite (auc = %0.3f)' % auc_tflite)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC - TFLite Model')
plt.legend(loc="lower right")
plt.savefig(model_prunned_save_path + "plot_roc_tflite_prunned.png")
plt.show()


In [ ]:
import tempfile

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  zipped_file = model_3xprunned_save_path + '3xPrunned.zip'
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
# print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
# print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
# print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Create a 10x smaller model

In [ ]:
model_10xprunned_save_path = save_path + "10xPrunned/"

# Verifica se o diretório existe, se não, cria o diretório
if not os.path.exists(model_10xprunned_save_path):
    os.makedirs(model_10xprunned_save_path)


converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

quantized_and_pruned_tflite_file = model_10xprunned_save_path + "10xPrunned_TFLite.tflite"

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

# print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

# Evaluate 10x smaller model

In [ ]:
import numpy as np
import time

def evaluate_model(interpreter, x_val, y_val):
    # Obter detalhes de entrada e saída do modelo TFLite
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_index = input_details[0]["index"]
    output_index = output_details[0]["index"]

    # Verificar tipo de dado esperado pelo modelo
    input_dtype = input_details[0]["dtype"]

    # Garantir que x_val tenha 4 dimensões (batch_size, height, width, channels)
    if len(x_val.shape) == 3:
        x_val = np.expand_dims(x_val, axis=-1)  # Adiciona dimensão do canal

    # Lista para armazenar as previsões
    prediction_classes = []

    print("Iniciando avaliação...")

    # Medir o tempo total de inferência (apenas dentro de interpreter.invoke())
    total_inference_time = 0.0

    for i, val_image in enumerate(x_val):
        if i % 10000 == 0 and i > 0:
            print(f"Avaliado em {i} amostras até agora.")

        # Pré-processamento: adicionar dimensão do batch e ajustar tipo de dado
        val_image = np.expand_dims(val_image, axis=0).astype(input_dtype)

        # Definir a entrada no modelo
        interpreter.set_tensor(input_index, val_image)

        # Medir o tempo de inferência
        start_inference = time.time()
        interpreter.invoke()
        end_inference = time.time()

        # Somar o tempo de inferência
        total_inference_time += (end_inference - start_inference)

        # Pós-processamento: obter a classe prevista
        output = interpreter.get_tensor(output_details[0]["index"])
        predicted_class = np.argmax(output[0])
        prediction_classes.append(predicted_class)

    print("Avaliação finalizada.")

    # Comparar as previsões com as classes reais para calcular a acurácia
    prediction_classes = np.array(prediction_classes)
    # Se y_val já for 1D, não use np.argmax
    if len(y_val.shape) == 1:  # Verifica se y_val é 1D
        y_val_classes = y_val
    else:  # Se for one-hot encoded
        y_val_classes = np.argmax(y_val, axis=1)
    accuracy = (prediction_classes == y_val_classes).mean()

    # Calcular o tempo médio de inferência por amostra
    num_samples = x_val.shape[0]
    inference_time_per_sample_us = (total_inference_time / num_samples) * 1e6  # Microssegundos

    print(f"Acurácia: {accuracy:.4f}")
    print(f"Tempo médio de inferência por amostra (somente invoke): {inference_time_per_sample_us:.2f} µs")

    return accuracy, inference_time_per_sample_us


In [ ]:
# Avaliar o modelo TFLite
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()


test_accuracy, inference_time = evaluate_model(interpreter, x_val, y_val)


# Exibir resultados
print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Inference time per sample for pruned model: {:.2f} microseconds'.format(inference_time))

In [ ]:
# file_name = f'log_modelo_fold3.txt'
# file1 = open(os.path.join(model_10xprunned_save_path, file_name), "a")

# start = time.time()
# y_pred_scores = model.predict(x_val)
# end = time.time()

# print(f"Runtime of the program is {end - start}", file=file1)
# total_time = end - start
# timesample = total_time/y_pred_scores.shape[0]
# print(f"us/sample is {timesample*1000000}", file=file1)

# y_pred_class = (y_pred_scores > 0.5).astype("int64")

# y_pred_scores2 = y_pred_scores
# y_pred_scores_0 = 1 - y_pred_scores
# y_pred_scores = np.concatenate([y_pred_scores_0, y_pred_scores], axis=1)

# accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_val, y_pred_class, y_pred_scores)


# mseTeste = mean_squared_error(y_val, y_pred_class)

# print(f'Results for Prunned Model [10x]:\n Recall of {recall}; accuracy of {accuracy}; precision of {precision}; f1 of {f1}; auroc of {auroc}; aupr of {aupr}', file=file1)

# sns.set(rc={'figure.figsize':(8,8)})
# plt.figure(figsize=(8,4))
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epochs')
# plt.legend(loc='upper right')
# plt.savefig(model_10xprunned_save_path + "loss_and_val_loss_prunned.png")
# plt.show()

# sns.set(rc={'figure.figsize':(8,8)})
# plt.figure(figsize=(8,4))
# plt.plot(history.history['accuracy'], label='Train accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation accuracy')
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epochs')
# plt.legend(loc='upper right')
# plt.savefig(model_10xprunned_save_path + "loss_and_val_accuracy_prunned.png")
# plt.show()

# # sns.set(rc={'figure.figsize':(8,8)})
# # subplot = skplt.metrics.plot_confusion_matrix(y_val, y_pred_class, normalize=True)
# # subplot.set_ylim(-0.5, 1.5)
# # plt.savefig(save_path + "conf_matrix.png")
# # plt.show()

# fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_val, y_pred_scores2)
# auc_keras = auc(fpr_keras, tpr_keras)


# sns.set(rc={'figure.figsize':(8,8)})
# plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
# plt.plot(fpr_keras, tpr_keras, marker='.', label='MLP (auc = %0.3f)' % auc_keras)
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC')
# plt.legend(loc="lower right")
# plt.savefig(model_10xprunned_save_path + "plot_roc_prunned.png")
# plt.show()
# #########################################################################################################
# #Plotando o histórico do treino
# pd.DataFrame(history.history).plot(figsize=(8, 5))
# plt.grid(True)
# plt.gca().set_ylim(0, 1)
# plt.show()

# file1.close()

# Create a more optimized model

In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import os

# **1. Configuração inicial e parâmetros gerais**
batch_size = 256  # Tamanho do lote para treinamento
epochs = 10       # Aumentar para garantir que a poda seja concluída
validation_split = 0.1  # Usar 10% dos dados para validação

# Calcular o número de passos para concluir a poda
num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# **2. Configuração da poda (pruning)**
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.7,  # Começa com 70% de sparsidade
        final_sparsity=0.95,   # Termina com 95% de sparsidade
        begin_step=0,          # Poda ativa desde o início
        end_step=end_step      # Finaliza após todas as épocas
    )
}

# Aplicar poda ao modelo
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
model_for_pruning = prune_low_magnitude(model, **pruning_params)

# **3. Compilar o modelo podado**
model_for_pruning.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# **4. Configurar callbacks**
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),  # Atualiza os pesos podados a cada passo
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=1e-5
    ),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),  # Interrompe cedo se não houver melhora
    tf.keras.callbacks.TensorBoard(log_dir='./logs/pruning', update_freq='batch')  # Logs para depuração
]

# **5. Treinamento do modelo podado**
history = model_for_pruning.fit(
    x_train, y_train,
    batch_size=batch_size,
    validation_split=validation_split,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1,
    shuffle=True
)

# **6. Remover wrappers de poda após treinamento**
# Isso prepara o modelo para exportação e uso em produção
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# **7. Exportar o modelo podado no formato Keras**
pruned_model_path = "./prunned_model/"
if not os.path.exists(pruned_model_path):
    os.makedirs(pruned_model_path)

pruned_keras_file = pruned_model_path + "Pruned_Model.h5"
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print("Modelo podado salvo em:", pruned_keras_file)

# **8. Converter o modelo podado para TFLite**
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)

# Ativar sparsidade otimizada no TFLite
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Converter para o formato TFLite
pruned_tflite_model = converter.convert()

# Salvar o modelo TFLite otimizado
pruned_tflite_file = pruned_model_path + "Pruned_Model.tflite"
with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print("Modelo TFLite podado salvo em:", pruned_tflite_file)

# **9. Verificar o tamanho do modelo**
def get_model_size(file_path):
    import os
    size = os.path.getsize(file_path)
    print(f"Tamanho do arquivo '{file_path}': {size / 1024:.2f} KB")
    return size

get_model_size(pruned_keras_file)
get_model_size(pruned_tflite_file)


Epoch 1/10


2024-11-25 21:16:55.903921: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1732580215.953236 1140595 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580215.982669 1140595 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580215.984477 1140595 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580215.989378 1140595 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580215.991353 1140595 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580216.005295 1140595 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580216.009717 1140595 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580216.012589 1140595 gpu_t

1809/1810 [============================>.] - ETA: 0s - loss: 0.0177 - accuracy: 0.9972

W0000 00:00:1732580293.934879 1140602 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580293.935836 1140602 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580293.936776 1140602 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580293.937778 1140602 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580293.938782 1140602 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580293.940542 1140602 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580293.942300 1140602 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580293.943973 1140602 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580293.945208 1140602 gp

1810/1810 [==============================] - ETA: 0s - loss: 0.0177 - accuracy: 0.9972

...

1810/1810 [==============================] - 84s 44ms/step - loss: 0.0177 - accuracy: 0.9972 - val_loss: 0.0189 - val_accuracy: 0.9965 - lr: 0.0010
Epoch 2/10
   3/1810 [..............................] - ETA: 1:11 - loss: 0.0283 - accuracy: 0.9935

W0000 00:00:1732580297.341171 1140603 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580297.341899 1140603 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580297.342483 1140603 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580297.343074 1140603 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580297.343659 1140603 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580297.344388 1140603 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580297.345129 1140603 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580297.345745 1140603 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732580297.346364 1140603 gp

1810/1810 [==============================] - 78s 43ms/step - loss: 0.0183 - accuracy: 0.9976 - val_loss: 0.0166 - val_accuracy: 0.9979 - lr: 0.0010
Epoch 3/10
1810/1810 [==============================] - 79s 43ms/step - loss: 0.0194 - accuracy: 0.9973 - val_loss: 0.0143 - val_accuracy: 0.9984 - lr: 0.0010
Epoch 4/10
1810/1810 [==============================] - 77s 43ms/step - loss: 0.0229 - accuracy: 0.9966 - val_loss: 0.0201 - val_accuracy: 0.9981 - lr: 0.0010
Epoch 5/10
1810/1810 [==============================] - 78s 43ms/step - loss: 0.0329 - accuracy: 0.9945 - val_loss: 0.0214 - val_accuracy: 0.9971 - lr: 0.0010
Epoch 6/10
1809/1810 [============================>.] - ETA: 0s - loss: 0.0614 - accuracy: 0.9872
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1810/1810 [==============================] - 78s 43ms/step - loss: 0.0614 - accuracy: 0.9872 - val_loss: 0.0241 - val_accuracy: 0.9980 - lr: 0.0010
Epoch 7/10
1810/1810 [==============================]

W0000 00:00:1732580833.857282 1138323 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1732580833.857293 1138323 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-11-25 21:27:13.857488: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpajw0v_ky
2024-11-25 21:27:13.858372: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-11-25 21:27:13.858382: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpajw0v_ky
2024-11-25 21:27:13.862814: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-11-25 21:27:13.863594: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-11-25 21:27:13.885939: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpajw0v_ky
2024-11-25 21:27:13.892705: I tensorflow/cc/saved_model/loader.cc

1368640

# Mais uma tentativa

In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import os

# **1. Configuração inicial**
batch_size = 256  # Tamanho do lote para treinamento
epochs = 10       # Número de épocas para garantir o efeito da poda
validation_split = 0.1  # Usar 10% dos dados para validação

# Calcular o número de passos para concluir a poda
num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# **2. Configuração da poda**
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.7,  # Começa com 70% de sparsidade
        final_sparsity=0.95,   # Termina com 95% de sparsidade
        begin_step=0,          # Poda ativa desde o início
        end_step=end_step      # Finaliza após todas as épocas
    )
}

# Aplicar poda estrutural ao modelo
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
model_for_pruning = prune_low_magnitude(model, **pruning_params)

# **3. Compilar o modelo**
model_for_pruning.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# **4. Configurar callbacks**
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),  # Atualiza os pesos podados a cada passo
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=1e-5
    ),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),  # Interrompe cedo se não houver melhora
    tf.keras.callbacks.TensorBoard(log_dir='./logs/pruning', update_freq='batch')  # Logs para depuração
]

# **5. Treinamento do modelo podado**
history = model_for_pruning.fit(
    x_train, y_train,
    batch_size=batch_size,
    validation_split=validation_split,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1,
    shuffle=True
)

# **6. Remover wrappers de poda após o treinamento**
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# **7. Reestruturar o modelo para remover pesos zerados**
def restructured_model(model):
    """Reestrutura o modelo, removendo pesos zerados e otimizando camadas."""
    restructured_layers = []
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
            # Apenas reestrutura camadas densas ou convolucionais
            weights, biases = layer.get_weights()
            weights = weights[weights != 0]  # Remove pesos zerados
            if weights.size > 0:  # Apenas adiciona camadas com pesos ativos
                restructured_layers.append(layer)
        else:
            # Adiciona outras camadas diretamente
            restructured_layers.append(layer)
    return tf.keras.Sequential(restructured_layers)

compact_model = restructured_model(model_for_export)

# **8. Exportar o modelo podado no formato Keras**
pruned_model_path = "./prunned_model_new/"
if not os.path.exists(pruned_model_path):
    os.makedirs(pruned_model_path)

pruned_keras_file = pruned_model_path + "Pruned_Model_new.h5"
compact_model.save(pruned_keras_file, include_optimizer=False)
print("Modelo podado salvo em:", pruned_keras_file)

# **9. Converter para TFLite sem quantização**
converter = tf.lite.TFLiteConverter.from_keras_model(compact_model)

# Conversão simples para TFLite
pruned_tflite_model = converter.convert()

# Salvar o modelo TFLite otimizado
pruned_tflite_file = pruned_model_path + "Pruned_Model_new.tflite"
with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print("Modelo TFLite podado salvo em:", pruned_tflite_file)

# **10. Verificar o tamanho do modelo**
def get_model_size(file_path):
    """Calcula o tamanho do modelo salvo."""
    size = os.path.getsize(file_path)
    print(f"Tamanho do arquivo '{file_path}': {size / 1024:.2f} KB")
    return size

get_model_size(pruned_keras_file)
get_model_size(pruned_tflite_file)


Epoch 1/10
1810/1810 [==============================] - 80s 44ms/step - loss: 0.0192 - accuracy: 0.9973 - val_loss: 0.0129 - val_accuracy: 0.9985 - lr: 0.0010
Epoch 2/10
1810/1810 [==============================] - 78s 43ms/step - loss: 0.0161 - accuracy: 0.9979 - val_loss: 0.0174 - val_accuracy: 0.9975 - lr: 0.0010
Epoch 3/10
1810/1810 [==============================] - 79s 43ms/step - loss: 0.0183 - accuracy: 0.9976 - val_loss: 0.0204 - val_accuracy: 0.9975 - lr: 0.0010
Epoch 4/10
1810/1810 [==============================] - ETA: 0s - loss: 0.0208 - accuracy: 0.9973
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1810/1810 [==============================] - 78s 43ms/step - loss: 0.0208 - accuracy: 0.9973 - val_loss: 0.0230 - val_accuracy: 0.9977 - lr: 0.0010
Epoch 5/10
1810/1810 [==============================] - 79s 43ms/step - loss: 0.0219 - accuracy: 0.9969 - val_loss: 0.0204 - val_accuracy: 0.9982 - lr: 5.0000e-04
Epoch 6/10
1810/1810 [================

W0000 00:00:1732582649.054961 1138323 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1732582649.054971 1138323 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-11-25 21:57:29.055069: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp9ag135hs
2024-11-25 21:57:29.055924: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-11-25 21:57:29.055935: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp9ag135hs
2024-11-25 21:57:29.060809: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-11-25 21:57:29.080900: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp9ag135hs
2024-11-25 21:57:29.087447: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 32378 microseconds.


5440596

# Mais uma tentativa

In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import os

# Configuração inicial
batch_size = 256
epochs = 10
validation_split = 0.1
num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Simplificação do modelo
def simplify_model(model):
    simplified_model = tf.keras.Sequential()
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D):
            simplified_model.add(tf.keras.layers.Conv2D(
                filters=max(8, layer.filters // 2),
                kernel_size=layer.kernel_size,
                strides=layer.strides,
                padding=layer.padding,
                activation=layer.activation
            ))
        elif isinstance(layer, tf.keras.layers.Dense):
            simplified_model.add(tf.keras.layers.Dense(
                units=max(16, layer.units // 2),
                activation=layer.activation
            ))
        else:
            simplified_model.add(layer)
    return simplified_model

model = simplify_model(model)

# Aplicar poda manualmente
def apply_pruning_to_layers(model):
    pruned_layers = []
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
            pruned_layers.append(tfmot.sparsity.keras.prune_low_magnitude(layer))
        else:
            pruned_layers.append(layer)
    return tf.keras.Sequential(pruned_layers)

model_for_pruning = apply_pruning_to_layers(model)

print("Aplicando poda ao modelo...")
# Configuração de poda
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.5,
        final_sparsity=0.9,
        begin_step=0,
        end_step=end_step
    )
}

print("Compilando...")
# Compilar o modelo
model_for_pruning.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Treinar o modelo com callbacks
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=1e-5
    ),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
]

print("Treinando o modelo...")
history = model_for_pruning.fit(
    x_train, y_train,
    batch_size=batch_size,
    validation_split=validation_split,
    epochs=epochs,
    callbacks=callbacks,
    shuffle=True
)

print("Removendo wrappers de poda...")
# Remover wrappers de poda
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# Converter para TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

pruned_tflite_model = converter.convert()

# Salvar modelo TFLite
pruned_model_path = "./optimized_prunned_model/"
if not os.path.exists(pruned_model_path):
    os.makedirs(pruned_model_path)

pruned_tflite_file = pruned_model_path + "Optimized_Prunned_Model.tflite"
with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print("Modelo TFLite podado e otimizado salvo em:", pruned_tflite_file)

Aplicando poda ao modelo...
Compilando...
Treinando o modelo...


AttributeError: 'PruneLowMagnitude' object has no attribute 'pruning_step'

: 